In [ ]:
!pip install flask-ngrok
#!pip install pyngrok
#!ngrok authtoken XXXX_my_authtoken_XXX

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, redirect, url_for, request, render_template, jsonify, Response
import re
from PIL import Image
from io import BytesIO
import base64
import numpy as np
import cv2

app = Flask(__name__, template_folder='/home/siyad/my_ML_Projects/hpe_inter/Lab14_local_deployment_onnx/templates', static_folder='/home/siyad/my_ML_Projects/hpe_inter/Lab14_local_deployment_onnx/static')
run_with_ngrok(app) 

@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')

@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        image_data = re.sub('^data:image/.+;base64,', '', request.json)
        pil_image = Image.open(BytesIO(base64.b64decode(image_data))).convert('RGB')  
        label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 
        label_names.sort()
        net = cv2.dnn.readNetFromONNX('//home/siyad/my_ML_Projects/hpe_inter/Lab14_local_deployment_onnx/model/cifar_classifier.onnx')
        img = cv2.resize(np.array(pil_image),(32,32))
        img = np.array([img]).astype('float64') / 255.0
        net.setInput(img)
        out = net.forward()
        index = np.argmax(out[0])
        label =  label_names[index].capitalize()
        return jsonify(result=label)
    return None

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Nov/2022 02:00:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2022 02:00:57] "GET /static/main.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2022 02:00:57] "GET /static/main.js HTTP/1.1" 304 -
127.0.0.1 - - [30/Nov/2022 02:00:58] "GET /favicon.ico HTTP/1.1" 404 -
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/siyad/miniconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/siyad/miniconda3/lib/python3.9/threading.py", line 1299, in run
    self.function(*self.args, **self.kwargs)
  File "/home/siyad/miniconda3/lib/python3.9/site-packages/flask_ngrok.py", line 70, in start_ngrok
    ngrok_address = _run_ngrok()
  File "/home/siyad/miniconda3/lib/python3.9/site-packages/flask_ngrok.py", line 31, in _run_ngrok
    ngrok = subprocess.Popen([executable, 'http', '5000'])
  File "/home/siyad/miniconda3/lib/python3.9/subprocess.py", line 951, in __ini